In [ ]:
%%capture --no-stderr
%pip install --upgrade --quiet langchain langchain-community sqlite3 
%pip install -qU langchain-google-genai
!pip install -qU langchain-
!pip install langchain-experimental

In [ ]:
import sqlite3
import pandas as pd

df = pd.read_excel('translated.xlsx')
df.head()

In [ ]:
df.rename(columns={'    Quantity' : 'Quantity'}, inplace=True)
df.info()

In [ ]:
import os
from langchain.utilities import SQLDatabase
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import display
import re

# Environment setup
if "GOOGLE_API_KEY" not in os.environ:
    from getpass import getpass
    os.environ["GOOGLE_API_KEY"] = getpass("Enter your Google AI API key: ")

# Initialize LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# Database setup
SQLITE_URI = "sqlite:///Material.db"
db = SQLDatabase.from_uri(SQLITE_URI, include_tables=['materials'], sample_rows_in_table_info=2)


def clean_sql_query(query: str) -> str:
    """Remove markdown formatting and extra whitespace from SQL query."""
    # Remove markdown code blocks
    query = re.sub(r'```sql\s*|\s*```', '', query)
    # Remove extra whitespace and newlines
    query = re.sub(r'\s+', ' ', query).strip()
    return query


def convert_to_sql_query(user_query: str) -> str:
    """Convert a natural language query into a structured SQL query."""
    system_message = """
    As a SQL expert, convert the following user input into a SQL query based on this schema:
    Table: materials
    Columns: Material, "Material Description", Component, Quantity

    Return only the SQL query without any markdown formatting or additional text.
    Ensure the SQL query is executable in a standard SQL environment.
    Use double quotes for column names with spaces.
    """
    messages = [
        SystemMessage(content=system_message),
        HumanMessage(content=f"User Query: {user_query}")
    ]
    sql_query = llm(messages).content.strip()
    print("Generated SQL Query:")
    display(sql_query)
    sql_query = clean_sql_query(sql_query)
    return sql_query

def retrieve_from_db(query: str) -> str:
    """Execute the SQL query and return results as a string."""
    try:
        result = db.run(query)
        print("Retrieved Data:")
        display(result)
        return str(result)
    except Exception as e:
        print(f"Database retrieval error: {e}")
        return "No data retrieved due to an error."

def generate_response(user_input: str) -> str:
    """Generate a response to the user's query using the RAG pipeline."""
    sql_query = convert_to_sql_query(user_input)
    db_result = retrieve_from_db(sql_query)
    
    system_message = """
    As a professional representative of a manufacturing and logistics company, 
    assist users in retrieving detailed product information based on their queries. 
    Use the provided database result to formulate the most relevant and accurate responses.
    """
    
    messages = [
        SystemMessage(content=system_message),
        HumanMessage(content=f"User Query: {user_input}\n\nDatabase Result: {db_result}\n\nPlease provide a detailed response based on this information:")
    ]
    
    response = llm(messages).content
    print("Final Response:")
    display(response)
    return response

# Example usage


In [ ]:
sample_query = "What are the components in the product with material code: OFSO2SBG?" 
response = generate_response(sample_query)
print(response)